In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import torch 
from argparse import Namespace
from tqdm import tqdm
import pickle 
import glob 
import ast

# Add the path to the directory containing the sybil module
sys.path.append('/workspace/home/tengyuezhang/sybil_cect/code/Sybil/')
from sybil.utils.metrics import concordance_index, get_survival_metrics
from sybil import Sybil, Serie
from sybil import visualize_attentions_v2

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
num_threads = os.cpu_count() // 5

In [11]:
IN_CASES_PATH = '/workspace/home/kkulkarni/Sybil/Results/lungx_Diagnosis_corrected.csv' 
OUT_FEATURE_PATH = '/workspace/home/tengyuezhang/sybil_cect/results/lungx/lungx_features.csv'
# # attention maps 
# OUT_VIS_DIR_PATH = '/workspace/home/tengyuezhang/sybil_cect/visualizations/lungx_attention_maps'
# SAVE_ATTN_MAPS = True 
# if SAVE_ATTN_MAPS and not os.path.exists(OUT_VIS_DIR_PATH):
#     os.makedirs(OUT_VIS_DIR_PATH)

In [4]:
# Initialize the Sybil model
model = Sybil("sybil_1")
num_years = 6

/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Running one example

In [14]:
# Load the CSV file
all_cases = pd.read_csv(IN_CASES_PATH)
all_cases

,AccessionNumber,SliceThickness,ConvolutionKernel,ImageOrientationPatient,Scan Number,index,Directory,Orientation,Year,timepoint,"Nodule Center x,y Position*",Nodule Center Image,Diagnosis
0,1791342422597581,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-BE001,0,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,405296.0,169.0,0
1,3027956620826907,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-BE002,1,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,184268.0,117.0,0
2,1764192885731917,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-BE006,2,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,449266.0,241.0,0
3,1110244576327898,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-BE007,3,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,385206.0,194.0,0
4,2472930474421214,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-BE010,4,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,120336.0,69.0,0
5,3138867356507502,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-LC001,5,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,120325.0,135.0,1
6,2773262718344969,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-LC002,6,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,139359.0,70.0,1
7,1365067556474585,1.0,D,"[1, 0, 0, 0, 1, 0]",CT-TRAINING-LC003,7,/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Trai...,axial,2007,0,375323.0,70.0,1
8,2771709101258922,1.0,D,"[1, 0, 0, 0, 1, 0]",LUNGX-CT001,10,/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-C...,axial,2006,0,"135, 303",142.0,0
9,1356118425455229,1.0,D,"[1, 0, 0, 0, 1, 0]",LUNGX-CT002,11,/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-C...,axial,2007,0,"330, 348",205.0,0


In [ ]:
example = all_cases.iloc[1]
example

In [7]:
dicom_dir = example['Directory']
event = 0
years_to_event = 1
pid = example['pid']
dicom_list = glob.glob(dicom_dir + '/*')
serie = Serie(dicom_list, label=event, censor_time=years_to_event)

results = model.predict([serie], return_attentions=True, threads=num_threads)

In [7]:
results 

Prediction(scores=[[0.012758719037714684, 0.025743208030524028, 0.040977454787905605, 0.05335478429725567, 0.06677989107741973, 0.10217879263786658]], attentions=[{'image_attention_1': array([[[[-11.470681, -10.07798 ,  -9.133754, ...,  -9.066321,
          -10.46895 , -12.185319],
         [-14.458355, -13.403002, -12.598696, ..., -12.14016 ,
          -14.100868, -15.431726],
         [-17.986788, -17.2131  , -16.881214, ..., -15.980452,
          -17.702059, -18.7915  ],
         ...,
         [-18.078018, -17.800737, -18.165924, ..., -17.937977,
          -18.665146, -19.214975],
         [-20.353813, -20.045208, -20.29802 , ..., -19.706808,
          -20.791616, -21.331861],
         [-16.724674, -16.201244, -16.207844, ..., -16.025623,
          -16.947964, -17.385353]]]], dtype=float32), 'volume_attention_1': array([[[-11.6407585, -11.967636 , -12.494825 , -12.254294 ,
         -11.702682 , -12.524836 , -11.818853 ,  -7.42701  ,
          -7.7739034,  -6.7461376,  -5.168269 ,  -

In [8]:
results.attentions[0]['features'][0, 0, :].tolist()

[-0.996924638748169,
 -0.24724116921424866,
 -0.1451576054096222,
 -0.4557493031024933,
 -0.40156227350234985,
 -1.0191634893417358,
 -0.34876951575279236,
 -0.5693208575248718,
 -0.17329853773117065,
 5.099407196044922,
 4.189969062805176,
 -0.12792639434337616,
 -0.2336098849773407,
 -0.08022262901067734,
 -0.4083849787712097,
 -0.392295777797699,
 -0.3131001889705658,
 0.04150423780083656,
 -0.16539950668811798,
 -1.0371801853179932,
 -0.47772982716560364,
 -0.2876025140285492,
 -0.043370623141527176,
 -0.033319566398859024,
 -0.08222568035125732,
 -0.1935325264930725,
 -0.08015225827693939,
 3.1899802684783936,
 0.9302515983581543,
 -0.28981462121009827,
 -2.9689841270446777,
 -0.795571506023407,
 -0.7838329076766968,
 -0.23678551614284515,
 -2.1063365936279297,
 -0.0017286315560340881,
 -0.6068207621574402,
 -2.451246738433838,
 1.7143911123275757,
 2.786296844482422,
 0.02837122417986393,
 -0.22373884916305542,
 -0.4724296033382416,
 -1.0213133096694946,
 -0.8410581350326538,
 -1

## On the entire dataset

In [15]:
# Load the CSV file
all_cases = pd.read_csv(IN_CASES_PATH)
df = all_cases 
df['pid'] = df['index']
num_features = 512 

In [20]:
for i in range(num_features):
    df[f'feature_{i}'] = np.nan 

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing cases"):
    dicom_dir = os.path.join('/workspace', row['Directory'][1:])
    print(dicom_dir)
    event = 0
    years_to_event = 1
    pid = row['pid']
    dicom_list = glob.glob(dicom_dir + '/*')
    serie = Serie(dicom_list, label=event, censor_time=years_to_event)
    
    # get predicted risk scores and features from the last hidden layer (returned along with the attentions)
    results = model.predict([serie], return_attentions=True, threads=num_threads)
    
    # append features 
    for i in range(num_features):
        # 'feature' for features before relu 
        # 'hidden' for features after relu
        df.at[index, f'feature_{i}'] = results.attentions[0]['features'][0, 0, i] 
        
    # save updated df 
    df.to_csv(OUT_FEATURE_PATH, index=False)

Processing cases:   0%|                                                                                                                                          | 0/54 [00:00<?, ?it/s]

/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Training-BE001/01-03-2007-16904-CT INFUSED CHEST-143.1/4.000000-HIGH RES-47.17
using device cuda:0


Processing cases:   2%|██▍                                                                                                                               | 1/54 [00:18<16:28, 18.65s/it]

/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/CT-Training-BE002/01-04-2007-6728-CT INFUSED CHEST-831.1/6.000000-HI-RES-98.22
using device cuda:0


Processing cases:   4%|████▊                                                                                                                             | 2/54 [00:39<17:11, 19.84s/it]

...

Processing cases:  96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏    | 52/54 [14:51<00:31, 15.90s/it]

/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT059/01-08-2007-6789-CT INFUSED CHEST-26028/5.000000-HI RES-62275
using device cuda:0


Processing cases:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 53/54 [15:05<00:15, 15.25s/it]

/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT060/02-20-2007-6367-CT INFUSED CHEST-13719/4.000000-HIGH RES-42551
using device cuda:0


Processing cases: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [15:21<00:00, 17.06s/it]
